### Setup the noisy simulator according to hardware calibration data

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService


# get a real backend from the runtime service
service = QiskitRuntimeService(channel="ibm_quantum", 
                               token="eff90e8bee70d824d8e6c145072c95ec9ba28369fd0c8aa5b192380f5c7ffc207f64bab13f2ae8897269ecdfa88b1f288c4e58c57bdcf86ed8bd500257170657", 
                               instance="ibm-q-ornl/ornl/csc509")

/var/folders/3t/7ts1c6s95hq650lfkvk66tkh0000gn/T/ipykernel_16092/655742877.py:5: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum",


In [2]:
service.backends()

[<IBMBackend('ibm_strasbourg')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibm_brussels')>,
 <IBMBackend('ibm_marrakesh')>,
 <IBMBackend('ibm_aachen')>,
 <IBMBackend('ibm_kingston')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_torino')>,
 <IBMBackend('ibm_fez')>]

Then, one can grab the desired backend.

In [5]:
device = service.backend('ibm_fez')

We can setup the noisy simulator by passing the backend properties to the aer simulator.

In [6]:
from qiskit_aer import AerSimulator

backend_sim = AerSimulator.from_backend(device)

---

In [ ]:
import time
import numpy as np
import random
import dill as pickle

import qiskit
from qiskit.circuit.library import RealAmplitudes
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector, state_fidelity


shots = 100000
num_trees = 333
num_qubits = 2
num_layers = 4
num_trials = 1
fid_sv_list = []
quantum_time_list = []
post_time_list = []
hf_sim = {}

print("=================================== Simulation Start =============================================")
for i in range(num_trials):
    print(f"=================================== Simulate {i}-th sample =========================================")
    ansatz = RealAmplitudes(num_qubits, entanglement='pairwise', reps=num_layers, skip_final_rotation_layer=True)
    params = loaded_param['parameter'][(num_qubits, num_layers)][0]
    list_circuits = get_circuits(num_qubits, ansatz)
    
    # record quantum time
    start_time = time.time()
    samples = get_samples_compile(num_qubits, sampler_noisy, list_circuits, shots, params, backend_sim)
    end_time = time.time()
    elapsed_time1 = end_time - start_time
    print("Quantum time for sampling:", elapsed_time1)
    quantum_time_list.append(elapsed_time1)

    # record post-processing time
    start_time = time.time()
    noisy_job_samp = get_statevector(num_qubits, num_trees, samples)
    end_time = time.time()
    elapsed_time2 = end_time - start_time
    print("Post-processing time for getting the signs:", elapsed_time2)
    post_time_list.append(elapsed_time2)
    
    noisy_job_samp = noisy_job_samp/np.linalg.norm(noisy_job_samp)

    # Bind parameters
    ansatz.assign_parameters(params, inplace=True)
    statevector = Statevector(ansatz)
    print(statevector[0])
    
    fid_noisy = state_fidelity(noisy_job_samp, statevector)
    fid_sv_list.append(fid_noisy)
    print("HForest FakeIBMQ Fidelity:", fid_noisy)
    print("Fidelity Upper Bound:", state_fidelity(np.abs(noisy_job_samp), np.abs(statevector)))
    negative_count = np.sum(statevector.data.real < 0)
    print(f"Qubits: {num_qubits} Layers:{ num_layers} positive_count: {100*np.sum(statevector.data.real[(statevector.data.real > 0)]**2)}%")
    print(f"Qubits: {num_qubits} Layers:{ num_layers} negative_count: {100*np.sum(statevector.data.real[(statevector.data.real < 0)]**2)}%")

    # Store data
    hf_sim.update({i:{
        "n_qubits": num_qubits,
        "n_shots": shots,
        "n_layers": num_layers,
        "n_trees": num_trees,
        "fid_state": fid_noisy,
        "time_quanutm": elapsed_time1,
        "time_postprocess": elapsed_time2,
        "samples": samples,
        "state": statevector,
        "state_hardware": noisy_job_samp,
        "fid_bound": state_fidelity(np.abs(noisy_job_samp), np.abs(statevector)),
        "negative_count": negative_count,
        "negative_percentage": 100*np.sum(statevector.data.real[(statevector.data.real < 0)]**2)}})

print("=================================== Simulation End =============================================")